In [38]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers.convolutional import Convolution1D
from keras.layers.convolutional import MaxPooling1D
import os
import glob
# fix random seed for reproducibility
numpy.random.seed(7)

In [2]:
def load_aclImdb_data(path_train, path_test):
    sentences_train = []
    train_y = []
    currdir = os.getcwd()
    os.chdir('%s/pos/' % path_train)
    for ff in glob.glob("*.txt"):
       	with open(ff, 'r') as f:
	    sentences_train.append(f.readline().strip())
            train_y.append(1)
    os.chdir('%s/neg/' % path_train)
    for ff in glob.glob("*.txt"):
       	with open(ff, 'r') as f:
            sentences_train.append(f.readline().strip())
            train_y.append(0)
    os.chdir(currdir)
    sentences_test = []
    test_y = []
    currdir = os.getcwd()
    os.chdir('%s/pos/' % path_test)
    for ff in glob.glob("*.txt"):
        with open(ff, 'r') as f:
            sentences_test.append(f.readline().strip())
            test_y.append(1)
    os.chdir('%s/neg/' % path_test)
    for ff in glob.glob("*.txt"):
        with open(ff, 'r') as f:
            sentences_test.append(f.readline().strip())
            test_y.append(0)
    os.chdir(currdir)
    return sentences_train, sentences_test, train_y, test_y
X_train, X_test, y_train, y_test = load_aclImdb_data("/Users/zacharie/Documents/AIC/REI-Option/aclImdb/train", "/Users/zacharie/Documents/AIC/REI-Option/aclImdb/test")

In [3]:
import string
from nltk.stem.porter import *
import multiprocessing
import unicodedata
import nltk
from joblib import Parallel, delayed
import tqdm

In [4]:
def stemmering_sentences(sentence):
    # Remove punctuation, stopword and then stemmering
    punctuation = set(string.punctuation)
    stemmer = nltk.PorterStemmer()
    tmp = unicode(sentence, errors='ignore')
    doc = [stemmer.stem(word.lower()) for word in nltk.word_tokenize(tmp) if
           (word not in punctuation) and (word not in nltk.corpus.stopwords.words('english')) and (word != 'br')]
    return doc

def stemmering_sentences_imdb(X, y):
    sentences_stem = Parallel(n_jobs=4)(delayed(stemmering_sentences)(sentence) for sentence in tqdm.tqdm(X, desc="stem"))
    return sentences_stem, y
X, y = stemmering_sentences_imdb(X_train+X_test, y_train+y_test)


stem: 100%|██████████| 50000/50000 [10:35<00:00, 78.64it/s]

stem: 100%|█████████▉| 49998/50000 [10:35<00:00, 66.21it/s]

stem: 100%|█████████▉| 49987/50000 [10:35<00:00, 63.42it/s]

stem: 100%|█████████▉| 49978/50000 [10:35<00:00, 74.71it/s]

stem: 100%|█████████▉| 49968/50000 [10:35<00:00, 86.82it/s]

...

stem:   0%|          | 66/50000 [00:00<10:47, 77.07it/s]

stem:   0%|          | 48/50000 [00:00<13:17, 62.66it/s]

stem:   0%|          | 24/50000 [00:00<17:28, 47.66it/s]

stem:   0%|          | 8/50000 [00:00<22:12, 37.51it/s]

stem:   0%|          | 0/50000 [00:00<?, ?it/s]

In [45]:
# import numpy as np
# print np.shape(X) == np.shape(y)
# print X[0]
# print X_train[0]

In [0]:
# combine_train_test_X = terms_by_doc_train + terms_by_doc_test
combine_train_test_X = X


In [83]:
import multiprocessing
# set parameters:
vocab_dim = 50
maxlen = 200
n_iterations = 1  # ideally more..
n_exposures = 30
window_size = 5
batch_size = 200
n_epoch = 2
cpu_count = multiprocessing.cpu_count()


In [84]:
from gensim.models.word2vec import Word2Vec
print('Training a Word2vec model...')
model = Word2Vec(size=vocab_dim,
                 min_count=n_exposures,
                 window=window_size,
                 workers=cpu_count,
                 iter=n_iterations)
model.build_vocab(combine_train_test_X)
model.train(combine_train_test_X)


5807713

Training a Word2vec model...


In [85]:
from gensim.corpora.dictionary import Dictionary
gensim_dict = Dictionary()
gensim_dict.doc2bow(model.vocab.keys(), allow_update=True)
# gensim_dict.items() returns [(0, u"'surpris"), (1, u'woodi'), (2, u'yellow'),...]
# K+1 aims at avoiding 0 as index.
w2indx = {v: k+1 for k, v in gensim_dict.items()}
w2vec = {word: model[word] for word in w2indx.keys()}
# print len(model["surpris"]) -> 100
print('Setting up Arrays for Keras Embedding Layer...')
n_symbols = len(w2indx) + 1  # adding 1 to account for 0th index
embedding_weights = np.zeros((n_symbols + 1, vocab_dim))
for word, index in w2indx.items():
    embedding_weights[index, :] = w2vec[word]


Setting up Arrays for Keras Embedding Layer...


In [86]:
from sklearn.model_selection import train_test_split
terms_by_doc_train, terms_by_doc_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=58)
X_train = []
for doc in terms_by_doc_train:
    new_txt = []
    for word in doc:
        try:
            new_txt.append(w2indx[word])
        except:
            new_txt.append(0)
    X_train.append(new_txt)
X_test = []
for doc in terms_by_doc_test:
    new_txt = []
    for word in doc:
        try:
            new_txt.append(w2indx[word])
        except:
            new_txt.append(0)
    X_test.append(new_txt)

In [87]:
print("Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

print('Convert labels to Numpy Sets...')
y_train = np.array(y_train)
y_test = np.array(y_test)

print('Defining a Simple Keras Model...')

model = Sequential()  # or Graph or whatever

model.add(Embedding(
    input_dim=n_symbols + 1,
    output_dim=vocab_dim,
    mask_zero=False,
    weights=[embedding_weights],
    input_length=maxlen))  # Adding Input Length

model.add(Convolution1D(nb_filter=32, filter_length=3, border_mode='same', activation='relu'))

model.add(MaxPooling1D(pool_length=2))

model.add(LSTM(vocab_dim))

model.add(Dropout(0.3))

model.add(Dense(1, activation='sigmoid'))

print('Compiling the Model...')
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              class_mode='binary',metrics=['accuracy'])

print("Train...")
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=n_epoch,
          validation_data=(X_test, y_test))

print("Evaluate...")
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)



('Test score:', 0.29072314838568369)
('Test accuracy:', 0.87973333279291788)


15000/15000 [==============================] - 10s    

14800/15000 [============================>.] - ETA: 0s

14600/15000 [============================>.] - ETA: 0s

14400/15000 [===========================>..] - ETA: 0s

...

  400/35000 [..............................] - ETA: 79s - loss: 0.7146 - acc: 0.4475

  200/35000 [..............................] - ETA: 81s - loss: 0.7275 - acc: 0.4150

Train on 35000 samples, validate on 15000 samples
Epoch 1/2


('X_train shape:', (35000, 200))
('X_test shape:', (15000, 200))
Convert labels to Numpy Sets...
Defining a Simple Keras Model...
Compiling the Model...
Train...


Pad sequences (samples x time)


In [134]:
from keras.models import load_model
model.save('lstm_train_imdb.h5')
import pickle
with open("./lstm_w2indx_dict_imdb.pkl", 'wb') as f:
    pickle.dump(w2indx, f)

In [135]:
with open("./lstm_w2indx_dict_imdb.pkl", 'rb') as f:
    w2indx_load = pickle.load(f)
model = load_model('lstm_train_imdb.h5')

In [132]:
demo = ["It is one good film.", "He said it is one good film, but he is wrong."]
demo = [stemmering_sentences(i) for i in demo]
X_demo = []
for doc in demo:
    new_txt = []
    for word in doc:
        try:
            new_txt.append(w2indx_load[word])
        except:
            new_txt.append(0)
    X_demo.append(new_txt)

In [133]:
X_demo = sequence.pad_sequences(X_demo, maxlen=maxlen)
print model.predict(X_demo)

[[ 0.60280693]
 [ 0.43718576]]
